In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version


# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop3.tgz'


Exception: Unable to find py4j in /content/spark-3.3.1-bin-hadoop3\python, your SPARK_HOME may not be configured correctly

In [ ]:
pip install pyspark

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales.show()


ModuleNotFoundError: No module named 'pyspark'

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView('home_sales')

NameError: name 'home_sales' is not defined

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
sql = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()

NameError: name 'spark' is not defined

In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT
  YEAR(date_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY YEAR DESC
""").show()

NameError: name 'spark' is not defined

In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT
  YEAR(date_built) AS YEAR_BUILT,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
and sqft_living >= 2000
and floors = 2
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT DESC
""").show()


NameError: name 'spark' is not defined

In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

sql = """
SELECT
  view, 
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined

In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')


NameError: name 'spark' is not defined

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

NameError: name 'spark' is not defined

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
sql = """
SELECT
  view, 
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))


NameError: name 'time' is not defined

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')


NameError: name 'home_sales' is not defined

In [13]:
# 11. Read the parquet formatted data.
p_home_sales = spark.read.parquet('p_home_sales')


NameError: name 'spark' is not defined

In [ ]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView('p_home_sales')


In [14]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
sql = """
SELECT
  view,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM p_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined

In [15]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

NameError: name 'spark' is not defined

In [16]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is still cached')
else:
  print('all clear')

NameError: name 'spark' is not defined